In [172]:
import pandas as pd
import numpy as np

In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [222]:
path = parentdir + "/final_models/"
dataset_names = ["cbf", "phalanges", "HARDataset", "EpilepticSeizureRecognition"]
dataset_names = ["esr", "har", "cbf", "poc"]
objects = ["blackboxes", "autoencoders"]

In [271]:
files = os.listdir(path)
csv_files = sorted([file for file in files if ".csv" in file])
df_dict = dict()
for csv in csv_files:
    df = pd.read_csv(path + csv, sep = ";", index_col = 0)
    df_dict[csv] = df
    

### Blackboxes

In [ ]:
# divido tabelle per metrica e metto solo test set

In [224]:
columns = ["RESNET", "CNN", "KNN"]
rows = ["ESR", "HAR", "CBF", "POC"]
rows = ["esr", "har", "cbf", "poc"]
#rows = ["EpilepticSeizureRecognition", "HumanActivityRecognition", "CylinderBellFunnel", "PhalangesOutileCorrect"]

In [259]:
accuracy_df = []
mse_df = []
for csv in df_dict:
    if "blackboxes" in csv:
        df = df_dict[csv]
        for column in df.columns:
            df[column] = [('%.3f' % (m)).replace('0.', '.') if m < 1.0 else '1.00' for m in df[column]]
        accuracy_df.append(df.loc["test_accuracy"].values)
        mse_df.append(df.loc["test_mse"].values)
accuracy_df = pd.DataFrame(accuracy_df, columns = columns, index = rows)
#accuracy_df.index.name = "DATASETS"
mse_df = pd.DataFrame(mse_df, columns = columns, index = rows)

final_dict = {"accuracy_df": accuracy_df, "mse_df": mse_df}

accuracy_df=accuracy_df.reset_index()
accuracy_df["metric"] = "acc"
mse_df=mse_df.reset_index()
mse_df["metric"] = "mse"
all_in_one_df = pd.concat([accuracy_df,mse_df]).set_index(['index', 'metric']).sort_values(by="index")

In [126]:
"""multi_index = []
for column in df.columns:
    for index in df.index:
        multi_index.append((column, index))
df_multi = pd.DataFrame([df.values.T.flatten()], columns = multi_index)
df_multi.columns = pd.MultiIndex.from_tuples(df_multi.columns)
df_multi[df_multi.columns[:6]]"""

'multi_index = []\nfor column in df.columns:\n    for index in df.index:\n        multi_index.append((column, index))\ndf_multi = pd.DataFrame([df.values.T.flatten()], columns = multi_index)\ndf_multi.columns = pd.MultiIndex.from_tuples(df_multi.columns)\ndf_multi[df_multi.columns[:6]]'

In [226]:
# separate df
for key in final_dict:
    df = final_dict[key]
    column_format = "|c|"
    for i in range(len(df.columns)):
        column_format += "c"
    column_format += "|"
    latex = df.to_latex(index = True, 
                               bold_rows = True,
                               column_format = column_format
                            #multicolumn_format = "c",
                            #multicolumn = True
                           )
    latex = latex.replace("toprule", "hline")
    latex = latex = latex.replace("midrule", "hline")
    latex = latex.replace("bottomrule", "hline")
    print("%{}".format(key))
    print(latex)
    print()
    print()
    print()

%accuracy_df
\begin{tabular}{|c|ccc|}
\hline
{} & RESNET &   CNN &   KNN \\
\hline
\textbf{esr} &   .978 &  .972 &  .865 \\
\textbf{har} &   .925 &  .891 &  .901 \\
\textbf{cbf} &   1.00 &  1.00 &  1.00 \\
\textbf{poc} &   .829 &  .768 &  .783 \\
\hline
\end{tabular}




%mse_df
\begin{tabular}{|c|ccc|}
\hline
{} & RESNET &   CNN &   KNN \\
\hline
\textbf{esr} &   .053 &  .081 &  .135 \\
\textbf{har} &   .266 &  .265 &  .146 \\
\textbf{cbf} &   .028 &  .350 &  .000 \\
\textbf{poc} &   .802 &  .489 &  .217 \\
\hline
\end{tabular}






In [286]:
df = all_in_one_df
column_format = "|c|c||""
for i in range(len(df.columns)):
    column_format += "c"
column_format += "|"
latex = df.to_latex(index = True, 
                    bold_rows = True,
                    column_format = column_format,
                    multirow = True)
latex = latex.replace("toprule", "hline")
latex = latex = latex.replace("midrule", "hline")
latex = latex.replace("bottomrule", "hline")
print("%{}".format(key))
print(latex)
print()
print()
print()

%phalanges_autoencoders_20191202_165821.csv
\begin{tabular}{|c|c||ccc|}
\hline
    &     & RESNET &   CNN &   KNN \\
\textbf{index} & \textbf{metric} &        &       &       \\
\hline
\multirow{2}{*}{\textbf{cbf}} & \textbf{acc} &   1.00 &  1.00 &  1.00 \\
    & \textbf{mse} &   .028 &  .350 &  .000 \\
\cline{1-5}
\multirow{2}{*}{\textbf{esr}} & \textbf{acc} &   .978 &  .972 &  .865 \\
    & \textbf{mse} &   .053 &  .081 &  .135 \\
\cline{1-5}
\multirow{2}{*}{\textbf{har}} & \textbf{acc} &   .925 &  .891 &  .901 \\
    & \textbf{mse} &   .266 &  .265 &  .146 \\
\cline{1-5}
\multirow{2}{*}{\textbf{poc}} & \textbf{acc} &   .829 &  .768 &  .783 \\
    & \textbf{mse} &   .802 &  .489 &  .217 \\
\hline
\end{tabular}






### Autoencoders

In [272]:

files = os.listdir(path)
csv_files = sorted([file for file in files if ".csv" in file])
df_dict = dict()
for csv in csv_files:
    df = pd.read_csv(path + csv, sep = ";", index_col = 0)
    df_dict[csv] = df
    

columns = ["RESNET", "CNN", "KNN"]
rows = ["AE-CNN", "VAE-CNN", "AAE-CNN", "AE-LSTM"]
final_dict = dict()
              
for csv in df_dict:
    if "autoencoders" in csv:
        df = df_dict[csv]
        df = df.loc[["reconstruction_resnet_test_accuracy",
                     "reconstruction_simplecnn_test_accuracy",
                     "reconstruction_knn_test_accuracy"]]
        for column in df.columns:
            df[column] = [('%.3f' % (m)).replace('0.', '.') if m < 1.0 else '1.00' for m in df[column]]
        final_dict[csv] = df
for dataset in final_dict:
    final_dict[dataset] = final_dict[dataset].T
    final_dict[dataset].columns = columns
    final_dict[dataset].index = rows

In [273]:
for key in final_dict:
    df = final_dict[key]
    column_format = "|c||"
    for i in range(len(df.columns)):
        column_format += "c"
    column_format += "|"
    latex = df.to_latex(index = True, 
                               bold_rows = True,
                               column_format = column_format
                            #multicolumn_format = "c",
                            #multicolumn = True
                           )
    latex = latex.replace("toprule", "hline")
    latex = latex = latex.replace("midrule", "hline")
    latex = latex.replace("bottomrule", "hline")
    print("%{}".format(key))
    print(latex)
    print()
    print()
    print()

%EpilepticSeizureRecognition_autoencoders_20200106_191206.csv
\begin{tabular}{|c||ccc|}
\hline
{} & RESNET &   CNN &   KNN \\
\hline
\textbf{AE-CNN } &   .955 &  .964 &  .959 \\
\textbf{VAE-CNN} &   .941 &  .959 &  .801 \\
\textbf{AAE-CNN} &   .923 &  .937 &  .869 \\
\textbf{AE-LSTM} &   .511 &  .502 &  .602 \\
\hline
\end{tabular}




%HARDataset_autoencoders_20191202_171636.csv
\begin{tabular}{|c||ccc|}
\hline
{} & RESNET &   CNN &   KNN \\
\hline
\textbf{AE-CNN } &   .950 &  .910 &  .968 \\
\textbf{VAE-CNN} &   .776 &  .771 &  .769 \\
\textbf{AAE-CNN} &   .946 &  .912 &  .955 \\
\textbf{AE-LSTM} &   .337 &  .170 &  .394 \\
\hline
\end{tabular}




%cbf_autoencoders_20191201_133423.csv
\begin{tabular}{|c||ccc|}
\hline
{} & RESNET &   CNN &   KNN \\
\hline
\textbf{AE-CNN } &   1.00 &  .889 &  1.00 \\
\textbf{VAE-CNN} &   1.00 &  1.00 &  1.00 \\
\textbf{AAE-CNN} &   1.00 &  1.00 &  1.00 \\
\textbf{AE-LSTM} &   .333 &  .333 &  .333 \\
\hline
\end{tabular}




%phalanges_autoencoders_201

In [274]:
df = final_dict["EpilepticSeizureRecognition_autoencoders_20200106_191206.csv"]

In [275]:
multi_index = []
for dataset in dataset_names:
    for column in df.columns:
        multi_index.append((dataset, column))
final_df = pd.DataFrame()
for dataset in final_dict.keys():
    final_df = pd.concat([final_df, final_dict[dataset]], axis = 1)
final_df.columns = multi_index
final_df.columns = pd.MultiIndex.from_tuples(final_df.columns)

In [216]:
# HORIZONTAL
df = final_df
column_format = "|c||"
for i in range(len(df.columns)):
    column_format += "c"
    if ((i + 1) % 3 == 0) and (i + 1 != len(df.columns)): column_format += "|"
column_format += "|"
latex = df.to_latex(index = True, 
                    bold_rows = True,
                    column_format = column_format,
                    multicolumn_format = "c",
                    )
latex = latex.replace("toprule", "hline")
latex = latex = latex.replace("midrule", "hline")
latex = latex.replace("bottomrule", "hline")
print("%{}".format(key))
print(latex)
print()
print()
print()

%phalanges_autoencoders_20191202_165821.csv
\begin{tabular}{|c||ccc|ccc|ccc|ccc|}
\hline
{} & \multicolumn{3}{c}{esr} & \multicolumn{3}{c}{har} & \multicolumn{3}{c}{cbf} & \multicolumn{3}{c}{poc} \\
{} & RESNET &   CNN &   KNN & RESNET &   CNN &   KNN & RESNET &   CNN &   KNN & RESNET &   CNN &   KNN \\
\hline
\textbf{AE-CNN } &   .955 &  .964 &  .959 &   .950 &  .910 &  .968 &   1.00 &  .889 &  1.00 &   .972 &  .972 &  .963 \\
\textbf{VAE-CNN} &   .941 &  .959 &  .801 &   .776 &  .771 &  .769 &   1.00 &  1.00 &  1.00 &   .639 &  .500 &  .648 \\
\textbf{AAE-CNN} &   .923 &  .937 &  .869 &   .946 &  .912 &  .955 &   1.00 &  1.00 &  1.00 &   .926 &  .981 &  .963 \\
\textbf{AE-LSTM} &   .511 &  .502 &  .602 &   .337 &  .170 &  .394 &   .333 &  .333 &  .333 &   .676 &  .769 &  .778 \\
\hline
\end{tabular}






In [221]:
# VERTICAL
df = final_df.T.sort_values(by="index")
column_format = "|c|c||"
for i in range(len(df.columns)):
    column_format += "c"
column_format += "|"
latex = df.to_latex(index = True, 
                    bold_rows = True,
                    #column_format = column_format,
                    multicolumn_format = "c",
                    multirow = True
                    )
latex = latex.replace("toprule", "hline")
latex = latex = latex.replace("midrule", "hline")
latex = latex.replace("bottomrule", "hline")
#print("%{}".format(key))
print(latex)
print()
print()
print()

\begin{tabular}{llllll}
\hline
    &     & AE-CNN & VAE-CNN & AAE-CNN & AE-LSTM \\
\hline
\multirow{3}{*}{\textbf{esr}} & \textbf{RESNET} &   .955 &    .941 &    .923 &    .511 \\
    & \textbf{CNN} &   .964 &    .959 &    .937 &    .502 \\
    & \textbf{KNN} &   .959 &    .801 &    .869 &    .602 \\
\cline{1-6}
\multirow{3}{*}{\textbf{har}} & \textbf{RESNET} &   .950 &    .776 &    .946 &    .337 \\
    & \textbf{CNN} &   .910 &    .771 &    .912 &    .170 \\
    & \textbf{KNN} &   .968 &    .769 &    .955 &    .394 \\
\cline{1-6}
\multirow{3}{*}{\textbf{cbf}} & \textbf{RESNET} &   1.00 &    1.00 &    1.00 &    .333 \\
    & \textbf{CNN} &   .889 &    1.00 &    1.00 &    .333 \\
    & \textbf{KNN} &   1.00 &    1.00 &    1.00 &    .333 \\
\cline{1-6}
\multirow{3}{*}{\textbf{poc}} & \textbf{RESNET} &   .972 &    .639 &    .926 &    .676 \\
    & \textbf{CNN} &   .972 &    .500 &    .981 &    .769 \\
    & \textbf{KNN} &   .963 &    .648 &    .963 &    .778 \\
\hline
\end{tabular}






### STABILITIES

In [334]:
path = parentdir + "/stabilities/"
dataset_names = ["cbf", "phalanges", "HARDataset", "EpilepticSeizureRecognition"]
dataset_names = ["esr", "har", "cbf", "poc"]
objects = ["blackboxes", "autoencoders"]
files = os.listdir(path)
csv_files = sorted([file for file in files if ".csv" in file])
df_dict = dict()
for csv in csv_files:
    df = pd.read_csv(path + csv, sep = ";", index_col = 0)
    df_dict[csv] = df
    

In [338]:
stability_shap_df = {"knn":[], "resnet":[], "cnn":[]}
rows = dataset_names
k = 30
metric = "mean"
for csv in sorted(df_dict.keys()):
    if "shap_" in csv:
        df = df_dict[csv]
        stab = df_dict[csv].loc[k][metric]
        stab = ('%.3f' % (stab)).replace('0.', '.') if stab < 1.0 else ('%.2f' % (stab)) if stab < 10 else ('%.1f' % (stab)) if stab < 100 else ('%i' % (stab))
        if "knn" in csv:
            stability_shap_df["knn"].append(stab)
        elif "simplecnn" in csv:
            stability_shap_df["cnn"].append(stab)
        else:
            stability_shap_df["resnet"].append(stab)

stability_shap_df = pd.DataFrame(stability_shap_df, index = rows)




In [339]:
stability_shap_df

,knn,resnet,cnn
esr,.971,.998,.983
har,.996,.995,1.00
cbf,.873,.899,.893
poc,.926,.976,.961


In [ ]:
### shapelets

In [355]:
path = parentdir + "/stabilities/"
dataset_names = ["cbf", "phalanges", "HARDataset", "EpilepticSeizureRecognition"]
dataset_names = ["esr", "har", "cbf", "poc"]
objects = ["blackboxes", "autoencoders"]
files = os.listdir(path)
npy_files = sorted([file for file in files if ".npy" in file if "shapelets" in file])
df_dict = dict()
for npy in npy_files:
    df = np.load(path + npy)
    df_dict[npy] = df

In [357]:
stability_shapelet_df = {"knn":[], "resnet":[], "cnn":[]}
rows = dataset_names
k = 30
metric = "mean"
for npy in sorted(df_dict.keys()):
    df = df_dict[npy]
    stab = df[:,0].mean()
    stab = ('%.3f' % (stab)).replace('0.', '.') if stab < 1.0 else ('%.2f' % (stab)) if stab < 10 else ('%.1f' % (stab)) if stab < 100 else ('%i' % (stab))
    if "knn" in npy:
        stability_shapelet_df["knn"].append(stab)
    elif "simplecnn" in npy:
        stability_shapelet_df["cnn"].append(stab)
    else:
        stability_shapelet_df["resnet"].append(stab)

#stability_shap_df = pd.DataFrame(stability_shap_df, index = rows)

In [358]:
stability_shapelet_df

{'knn': ['.129'], 'resnet': ['.277'], 'cnn': []}